# Lista Prodotti DOP - Unione Europea

La lista dei prodotti DOP nell'Unione Europea è aggiornata dal Dipartimento per l'Agricoltura e lo Sviluppo Rurale.
http://ec.europa.eu/agriculture/quality/door/list.html?locale=it

http://nbviewer.jupyter.org/gist/jtbaker/57a37a14b90feeab7c67a687c398142c?flush_cache=true


In [201]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium #https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples
import json
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
url = 'http://ec.europa.eu/agriculture/quality/door/Denominations.xls?locale'
df = pd.read_excel(url, skiprows = 3, header=0)

In [4]:
df.head(2)

,Dossier Number,Designation,Country,ISO,Status,Type,Last relevant date,Product Categrory,Latin Transcription,Submission date,Publication date,Registration date,1st Amendment date,2nd Amendment date,3rd Amendment date
0,HR/PGI/0005/02179,Lička janjetina,Croatia,HR,Registered,PGI,31/10/2018,Class 1.1. Fresh meat (and offal),NaN,13/09/2016,02/05/2018,31/10/2018,NaN,NaN,NaN
1,PGI/IT/0005/02313,Lucanica di Picerno,Italy,IT,Registered,PGI,29/10/2018,"Class 1.2. Meat products (cooked, salted, smok...",NaN,05/06/2017,02/05/2018,29/10/2018,NaN,NaN,NaN


In [5]:
print (df.columns.tolist())

['   Dossier Number  ', '      Designation        ', ' Country ', ' ISO ', '   Status   ', '   Type    ', ' Last relevant date ', '   Product Categrory   ', '      Latin Transcription     ', ' Submission date ', ' Publication date ', ' Registration date ', ' 1st Amendment date ', ' 2nd Amendment date ', ' 3rd Amendment date ']


In [6]:
def desc_ita(x):
    if x['   Type    ']=='PDO':
        return 'DOP'
    elif x['   Type    ']=='PGI':
        return 'IGP'
    elif x['   Type    ']=='TSG':
        return 'STG'

In [7]:
df['Type IT'] = df.apply(lambda x: desc_ita(x), axis=1)

In [8]:
df.head(2)

,Dossier Number,Designation,Country,ISO,Status,Type,Last relevant date,Product Categrory,Latin Transcription,Submission date,Publication date,Registration date,1st Amendment date,2nd Amendment date,3rd Amendment date,Type IT
0,HR/PGI/0005/02179,Lička janjetina,Croatia,HR,Registered,PGI,31/10/2018,Class 1.1. Fresh meat (and offal),NaN,13/09/2016,02/05/2018,31/10/2018,NaN,NaN,NaN,IGP
1,PGI/IT/0005/02313,Lucanica di Picerno,Italy,IT,Registered,PGI,29/10/2018,"Class 1.2. Meat products (cooked, salted, smok...",NaN,05/06/2017,02/05/2018,29/10/2018,NaN,NaN,NaN,IGP


In [43]:
# Filtro solo i registrati e aggrego per Paese
df=df[(df['   Status   ']=='Registered')]
df_agg = df.groupby([' ISO '],as_index=False).count()

In [52]:
df_eu = df_agg[[' ISO ','   Dossier Number  ']].sort_values(['   Dossier Number  '], ascending=False)
df_eu.head(2)

,ISO,Dossier Number
21,IT,299
13,FR,248


# Data Visualization
Utilizzo GeoPandas

In [212]:
gdf = gpd.read_file("euro.geojson")
gdf1=gdf.merge(df_eu, left_on='wb_a2', right_on=' ISO ', how='left').fillna(0)
gdf1.head(2)

,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,type,admin,adm0_a3,...,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart,geometry,ISO,Dossier Number
0,1,Admin-0 country,6,Albania,ALB,0,2,Sovereign country,Albania,ALB,...,Southern Europe,Europe & Central Asia,7,7,4,-99,1,"POLYGON ((20.59024743010491 41.85540416113361,...",0,0.0
1,1,Admin-0 country,4,Austria,AUT,0,2,Sovereign country,Austria,AUT,...,Western Europe,Europe & Central Asia,7,7,5,-99,1,"POLYGON ((16.97966678230404 48.12349701597631,...",AT,17.0


In [211]:
centroid=gdf1.geometry.centroid

In [247]:
from branca.colormap import linear

colormap = linear.YlGn_09.scale(
    gdf1['   Dossier Number  '].min(),
    gdf1['   Dossier Number  '].max())

colormap.caption = "Numero Prodotti DOP, IGP e STG"

In [252]:
# mappa
m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=4)

# layer 1
folium.GeoJson(gdf1[['geometry',' ISO ','   Dossier Number  ','sovereignt']],
               name="Prodotti DOP, IGP e STG",
               style_function=lambda x: {
                       "weight":2, 
                       'color':'black',
                       'fillColor':colormap(x['properties']['   Dossier Number  ']), 
                       'fillOpacity':0.5},
               highlight_function=lambda x: {'weight':3, 'color':'black'},
               smooth_factor=2.0,
               tooltip = folium.features.GeoJsonTooltip(
                                              fields=['sovereignt','   Dossier Number  ',],
                                              aliases=['Paese','N° Prodotti'], 
                                              labels=True, 
                                              sticky=True
                   )
              ).add_to(m)

folium.LayerControl().add_to(m)
colormap.add_to(m)

m.save('FoodPorn_Prodotti_EU.html')

m